In [1]:
import pandas as pd
from music21 import converter, instrument, note, chord
from sklearn.preprocessing import LabelEncoder
import itertools

In [7]:
path = 'data\maestro-v3.0.0.csv'
df = pd.read_csv(path)

In [8]:
df = df.sort_values(by='duration').iloc[:10]

In [9]:
#Cration du tableau contenant les musiques sous forme de note
def extract_notes(file):
    midi = converter.parse(file)
    notes = []
    for part in midi.parts:
        for element in part.flat.notes:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

def ex_notes_tab(df,col_out):
    df['midi_file_data'] = 'data/'+df['midi_filename']
    df[col_out] = df['midi_file_data'].apply(extract_notes)
    return df

In [10]:
#Encodage
def encodage(df,col):
    all_notes = []
    for k in df[col].to_list():
        all_notes+=k
    all_notes = list(set(all_notes))

    encoder = LabelEncoder()
    encoder.fit(all_notes)

    df['encoded'] = df[col].apply(encoder.transform)
    return df

In [11]:
def colonnes(df):
    df_expanded = df['X'].apply(pd.Series)
    df_expanded['y'] = df['y']

    return df_expanded

In [12]:
#préparation tableau
def sequence_list(L):
    sequence_length = 500

    input_sequences = []
    output_notes = []

    for i in range(len(L) - sequence_length):
        seq_in = L[i:i + sequence_length]  
        seq_out = L[i + sequence_length]  
        input_sequences.append(seq_in)
        output_notes.append(seq_out)

    return input_sequences,output_notes

def sequence(dff):
    X,y = [],[]
    for k in dff['encoded'].to_list():
        xtemp,ytemp = sequence_list(k)
        X.append(xtemp)
        y.append(ytemp)
    X=list(itertools.chain(*X))
    y=list(itertools.chain(*y))

    return pd.DataFrame({"X":X,"y":y})
    

In [13]:
def preprocessing(df):
    print("extraction notes")
    df = ex_notes_tab(df,'notes')
    notess = df['notes'].to_list()
    print("encodage")
    df = encodage(df,'notes')

    print("sequence train")
    train = colonnes(sequence(df[df['split']=='train']))
    print("sequence test")
    test = colonnes(sequence(df[df['split']=='test']))
    print("sequence val")
    val = colonnes(sequence(df[df['split']=='validation']))

    return train,test,val,notess

In [15]:
train,test,val,notess = preprocessing(df)

extraction notes


C:\Users\ccdeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\music21\stream\base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


encodage
sequence train
sequence test
sequence val


In [16]:
train

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,y
0,136,212,210,55,268,371,311,498,210,311,...,81,232,354,219,254,64,254,5,236,354
1,212,210,55,268,371,311,498,210,311,268,...,232,354,219,254,64,254,5,236,354,207
2,210,55,268,371,311,498,210,311,268,55,...,354,219,254,64,254,5,236,354,207,330
3,55,268,371,311,498,210,311,268,55,136,...,219,254,64,254,5,236,354,207,330,68
4,268,371,311,498,210,311,268,55,136,210,...,254,64,254,5,236,354,207,330,68,236
5,371,311,498,210,311,268,55,136,210,268,...,64,254,5,236,354,207,330,68,236,62
6,311,498,210,311,268,55,136,210,268,55,...,254,5,236,354,207,330,68,236,62,235
7,498,210,311,268,55,136,210,268,55,124,...,5,236,354,207,330,68,236,62,235,282
8,210,311,268,55,136,210,268,55,124,170,...,236,354,207,330,68,236,62,235,282,419
9,311,268,55,136,210,268,55,124,170,10,...,354,207,330,68,236,62,235,282,419,270
